<a href="https://colab.research.google.com/github/musman7888/Generative-AI-Projects---Batch-58/blob/main/LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

In [ ]:
!pip install -qU pypdf

In [3]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [17]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "quickstart9"
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [32]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("documents.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt'}, page_content='Usman, born on 10th December, 1995 in Mandi'), Document(metadata={'source': 'documents.txt'}, page_content='Bahauddin, Pakistan. Did bachelors from'), Document(metadata={'source': 'documents.txt'}, page_content='from University of Sargodha. Now doing job in'), Document(metadata={'source': 'documents.txt'}, page_content='in Islamabad and also now living in Islamabad.')]


In [ ]:
!pip install tqdm

In [33]:
from tqdm import tqdm
vectors=[]
i = 1
ids = "ID No:"
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    doc_id = str(f"{ids}{i}")
    index.upsert(vectors=[(doc_id, vector, {'text': doc.page_content})])
    i=i+1

100%|██████████| 4/4 [00:01<00:00,  2.48it/s]


In [34]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

In [35]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [37]:
query = "usman complete study from where?"
response = qa_chain.run(query)
print(response)


The provided context doesn't contradict the original answer.  Therefore, the original answer remains: Usman completed his studies at the University of Sargodha.

